In [1]:
#Aquiagregamos todas las librerias
import numpy as np
import math


In [2]:
def particle_swarm_optimization(obj_func, dim, bounds, num_particles, max_iter, inertia, cognitive, social):
    # Inicializar partículas
    positions = np.random.uniform(bounds[0], bounds[1], (num_particles, dim))
    velocities = np.random.uniform(-1, 1, (num_particles, dim))
    personal_best_positions = positions.copy()
    personal_best_scores = np.array([obj_func(pos) for pos in positions])
    global_best_position = personal_best_positions[np.argmin(personal_best_scores)]
    global_best_score = np.min(personal_best_scores)

    # PSO Loop
    for iter in range(max_iter):
        for i in range(num_particles):
            # Evaluar la posición actual
            fitness = obj_func(positions[i])
            
            # Actualizar mejor personal
            if fitness < personal_best_scores[i]:
                personal_best_scores[i] = fitness
                personal_best_positions[i] = positions[i]
            
            # Actualizar mejor global
            if fitness < global_best_score:
                global_best_score = fitness
                global_best_position = positions[i]

        # Actualizar velocidades y posiciones
        for i in range(num_particles):
            r1, r2 = np.random.rand(dim), np.random.rand(dim)  # Factores aleatorios
            cognitive_component = cognitive * r1 * (personal_best_positions[i] - positions[i])
            social_component = social * r2 * (global_best_position - positions[i])
            velocities[i] = inertia * velocities[i] + cognitive_component + social_component
            positions[i] = positions[i] + velocities[i]
            
            # Limitar posiciones a los límites dados
            positions[i] = np.clip(positions[i], bounds[0], bounds[1])

        # Imprimir progreso
        #print(f"Iteración {iter + 1}/{max_iter}, Mejor Fitness: {global_best_score}")

    return global_best_position, global_best_score


In [3]:
# Algoritmo Gradient Evolution (GE)
def gradient_evolution(objective_function, dim, bounds, num_individuals, max_iter, c1, Jr, Sr, epsilon):
    # Paso 1: Inicialización
    population = np.random.uniform(bounds[0], bounds[1], (num_individuals, dim))
    fitness = np.array([objective_function(ind) for ind in population])
    best_index = np.argmin(fitness)
    global_best = population[best_index].copy()
    global_best_fitness = fitness[best_index]
    updating_history = np.ones(num_individuals)
    c=np.linspace(c1[0],c1[1] , max_iter)
    # Iteraciones del algoritmo
    for iteration in range(max_iter):
        new_population = population.copy()

        for i in range(num_individuals):
            # Identificar los mejores y peores individuos
            best_neighbor = population[np.argmin(fitness)]
            worst_neighbor = population[np.argmax(fitness)]

            # Actualización de vectores
            if np.array_equal(population[i], global_best):  # Si es el mejor
                delta = c[iteration] + (np.abs(worst_neighbor - population[i])) / 2
                bj = population[i] - delta
                new_population[i] = population[i]-(np.random.uniform(0, 1, dim)*(delta/2))*((worst_neighbor - bj)/(worst_neighbor-population[i]+ bj))+np.random.uniform(0, 1, dim) * (global_best-population[i])
            elif np.array_equal(population[i], worst_neighbor):  # Si es el peor
                delta = c[iteration] + (np.abs(population[i]-best_neighbor )) / 2
                wj = population[i] + delta
                new_population[i] =population[i]-(np.random.uniform(0, 1, dim)*(delta/2))*((wj-best_neighbor)/(wj-population[i]+ best_neighbor))+np.random.uniform(0, 1, dim) * (global_best-population[i])
            else:  # Otros individuos
                delta = np.abs(population[i] - best_neighbor) / 2 + np.abs(worst_neighbor - population[i]) / 2
                new_population[i] = population[i] - (np.random.uniform(0, 1, dim)*(delta/2))*((worst_neighbor-best_neighbor)/(worst_neighbor-population[i]+ best_neighbor))+np.random.uniform(0, 1, dim)*(global_best-population[i])

            # Vector Jumping
            if np.random.rand() < Jr:
                new_population[i] = new_population[i]+np.random.normal(0, 1, dim) * (new_population[i] - population[i])

            # Calcular aptitud
            new_fitness = objective_function(new_population[i])

            # Vector Refreshing
            if new_fitness < fitness[i]:
                fitness[i] = new_fitness
                population[i] = new_population[i]
            else:
                updating_history[i] -= epsilon * updating_history[i]
                if updating_history[i] < Sr:
                    population[i] = np.random.uniform(bounds[0], bounds[1], dim)
                    fitness[i] = objective_function(population[i])
                    updating_history[i] = 1

        # Actualizar el mejor vector global
        best_index = np.argmin(fitness)
        if fitness[best_index] < global_best_fitness:
            global_best = population[best_index].copy()
            global_best_fitness = fitness[best_index]

        # Imprimir progreso
        #print(f"Iteración {iteration + 1}/{max_iter}, Mejor Fitness: {global_best_fitness}")

    return global_best, global_best_fitness

In [4]:
#En este modulo se insertan todasla funciones para testear que aparecen en tabla 1 de 
#The gradient evolution algorithm: A new metaheuristic
def Sphere(x):
    return sum([xi**2 for xi in x])
def WeightedSphere(x):
    return sum([(i+1)*xi**2 for i, xi in enumerate(x)])
def SumOfDifferentPower(x):
    return sum([abs(xi) ** (i + 1) for i, xi in enumerate(x)])
def StepFunction (x):
    return sum([math.floor(xi+0.5)**2 for xi in x])
def Schwefel1_2(x):
    sum_j = 0
    for i in range(1, len(x) + 1):
        sum_i = sum(x[:i])#Suma hasta el elemento i
        sum_j += sum_i**2
    return sum_j
def Schwefel2_2(x):
    return sum([abs(xi) for xi in x])+np.prod([abs(xi) for xi in x])
def rosenbrock(x):
    sum_i = 0
    #Se toma D-1 para que la función pueda quedar dentro de los limites del indice
    for i in range(len(x) - 1):
        sum_i += (x[i] - 1)**2 + 100 * (x[i+1] - x[i]**2)**2
    return sum_i
def easom(x):
    return -((-1)**len(x)) * np.prod(np.cos(x)**2)*np.exp(-np.sum((x - np.pi)**2))
def ackley(x):
    D = len(x)
    suma_cuadrados = sum([xi**2 for xi in x])
    suma_cosenos=sum([np.cos(2 * np.pi * xi) for xi in x])
    term1 = -20 * np.exp(-0.2 * np.sqrt(suma_cuadrados / D))
    term2 = -np.exp(suma_cosenos / D)
    return term1 + term2 + 20 + np.exp(1)
def griewank(x):
    suma_cuadrados = sum([xi**2 for xi in x])
    producto_cosenos = np.prod([np.cos(xi / np.sqrt(i+1)) for i, xi in enumerate(x)])
    return 1/4000*suma_cuadrados-producto_cosenos+1
def rastrigin(x):
    return sum([(xi**2 - 10 * np.cos(2 * np.pi * xi))+10 for xi in x])
def salomon(x):
    suma_cuadrados = sum([xi**2 for xi in x])
    return - np.cos(2 * np.pi * np.sqrt(suma_cuadrados)) + 0.1 * np.sqrt(suma_cuadrados)+1
def schwefel_2_3(x):
    return 418.9892*len(x)*sum([-xi*np.sin(np.sqrt(abs(xi))) for xi in x]) 
def Whitley(x):
    D = len(x)
    total_sum = 0
    for i in range(D):
        for j in range(D):
            y_ij = 100 * (x[j] - x[i]**2)**2 + (1 - x[j])**2
            total_sum += (y_ij**2 / 4000 - np.cos(y_ij) + 1)**2
    return total_sum

In [5]:
dim = 10  # Dimensión del problema
bounds = (-100, 100)  # Límites del espacio de búsqueda
num_particles = 20  # Número de partículas
max_iter = 3000  # Número de iteraciones
inertia = 0.7  # Factor de inercia
cognitive = 1.5  # Componente cognitivo
social = 1.5  # Componente social

In [6]:
best_position, best_score = particle_swarm_optimization(
    obj_func=Sphere,
    dim=dim,
    bounds=bounds,
    num_particles=num_particles,
    max_iter=max_iter,
    inertia=inertia,
    cognitive=cognitive,
    social=social
)

# Resultados
print("Mejor posición encontrada:", best_position)
print("Fitness de la mejor posición:", best_score)

Mejor posición encontrada: [-3.86331964e-35  2.02732348e-34  1.19691665e-34  8.44416189e-35
 -1.59564876e-34  8.78855557e-35  1.94493293e-34 -3.79720042e-35
 -4.62532078e-35  1.20883819e-34]
Fitness de la mejor posición: 1.2971884765862566e-67


Para una esfera 10 dimensional, una población de 20 y 300 iteraciones la mejor aproximaciòn fue de 1.03E-128 usando el algoritmo de DE,  con el algoritmo DE PSO En el paper tiene como mejor aproximación 1.08E-42,  

Para la función de Rastrigin
$$f(\mathbf{x}) = \sum_{i=1}^D \left( x_i^2 - 10 \cos(2\pi x_i) + 10 \right)$$
una población de 20 y 300 iteraciones maximas la mejor aproximaciòn fue de 0 con el algoritmo de GE, usando el algoritmo DE PSO En el paper tiene como mejor aproximación 2.96.  

In [18]:

best_position, best_score = particle_swarm_optimization(
    obj_func=rastrigin,
    dim=dim,
    bounds=bounds,
    num_particles=num_particles,
    max_iter=max_iter,
    inertia=inertia,
    cognitive=cognitive,
    social=social
)

# Resultados
print("Mejor posición encontrada:", best_position)
print("Fitness de la mejor posición:", best_score)

Mejor posición encontrada: [-9.94958638e-01  1.98991223e+00 -9.94958639e-01  1.52920952e-09
 -3.66081732e-10  2.98485570e+00 -1.98991223e+00 -2.23869269e-10
 -9.94958635e-01  6.97085830e-11]
Fitness de la mejor posición: 19.899140793875056


In [8]:
best_position, best_score = particle_swarm_optimization(
    obj_func=WeightedSphere,
    dim=dim,
    bounds=bounds,
    num_particles=num_particles,
    max_iter=max_iter,
    inertia=inertia,
    cognitive=cognitive,
    social=social
)

# Resultados
print("Mejor posición encontrada:", best_position)
print("Fitness de la mejor posición:", best_score)

Mejor posición encontrada: [ 4.76271178e-34  4.33840890e-34  1.21648093e-34  2.07739276e-34
 -1.93789002e-34  4.15132965e-35 -2.30983220e-34  1.20764982e-34
 -6.75474756e-35  2.01544352e-34]
Fitness de la mejor posición: 9.093179477301123e-67


In [17]:

# Parámetros del algoritmo
dim = 10  # Dimensiones del problema
bounds = (-100, 100)  # Límites del espacio de búsqueda
num_individuals = 20  # Tamaño de la población
max_iter = 3000  # Número de iteraciones
c = [0.5,0.1]  # Parámetro de ajuste, en caso de buscarlo fijo colocar el mismo valor en ambos parametros, para mejor resultado en orden descendiente 
Jr = 0.5  # Probabilidad de salto vectorial
Sr = 0.1  # Umbral de refresco
epsilon = 0.05  # Factor de reducción de historia

# Ejecutar el algoritmo GE para esfera
best_position, best_fitness = gradient_evolution(
    Sphere, dim, bounds, num_individuals, max_iter, c, Jr, Sr, epsilon
)
# Resultados
print("Mejor posición encontrada para Sphere:", best_position)
print("Fitness de la mejor posición para Sphere:", best_fitness)
# Ejecutar el algoritmo GE para rastrigin
best_position, best_fitness = gradient_evolution(
    rastrigin, dim, bounds, num_individuals, max_iter, c, Jr, Sr, epsilon
)
# Resultados
print("Mejor posición encontrada para rastrigin:", best_position)
print("Fitness de la mejor posición para rastrigin:", best_fitness)

Mejor posición encontrada para Sphere: [-0.02825952 -0.0192847  -0.01511902 -0.01051697 -0.00447679 -0.01450011
 -0.01075858 -0.01576405 -0.00650913 -0.01955452]
Fitness de la mejor posición para Sphere: 0.002528986717812651
Mejor posición encontrada para rastrigin: [-2.00873386  1.01478876  0.10765467 -1.02923727 -2.0762634  -2.03877808
 -1.09885604 -0.06794322 -1.06539094  0.96554032]
Fitness de la mejor posición para rastrigin: 25.563456464951507


In [21]:
# Parámetros del algoritmo
dim = 10  # Dimensiones del problema
bounds = (-100, 100)  # Límites del espacio de búsqueda
num_individuals = 20  # Tamaño de la población
max_iter = 3000  # Número de iteraciones
c = [0.8,0.3]  # Parámetro de ajuste, en caso de buscarlo fijo colocar el mismo valor en ambos parametros, para mejor resultado en orden descendiente 
Jr = 0.5  # Probabilidad de salto vectorial
Sr = 0.1  # Umbral de refresco
epsilon = 0.05  # Factor de reducción de historia

# Ejecutar el algoritmo GE para WeightedSphere
best_position, best_fitness = gradient_evolution(
    WeightedSphere, dim, bounds, num_individuals, max_iter, c, Jr, Sr, epsilon
)
# Resultados
print("Mejor posición encontrada para 2 WeightedSphere:", best_position)
print("Fitness de la mejor posición para 2 WeightedSphere:", best_fitness)
# Ejecutar el algoritmo GE para Schwefel1_2
best_position, best_fitness = gradient_evolution(
    Schwefel1_2, dim, bounds, num_individuals, max_iter, c, Jr, Sr, epsilon
)
# Resultados
print("Mejor posición encontrada para 5 Schwefel1_2:", best_position)
print("Fitness de la mejor posición para 5 Schwefel1_2:", best_fitness)


# Ejecutar el algoritmo GE para salomon
best_position, best_fitness = gradient_evolution(
    salomon, dim, bounds, num_individuals, max_iter, c, Jr, Sr, epsilon
)
# Resultados
print("Mejor posición encontrada para 12 salomon:", best_position)
print("Fitness de la mejor posición para 12 salomon:", best_fitness)

# Ejecutar el algoritmo GE para easom
bounds = (-np.pi, np.pi)
best_position, best_fitness = gradient_evolution(
    easom, dim, bounds, num_individuals, max_iter, c, Jr, Sr, epsilon
)
# Resultados
print("Mejor posición encontrada para 8 easom:", best_position)
print("Fitness de la mejor posición para 8 easom:", best_fitness)

Mejor posición encontrada para 2 WeightedSphere: [-0.13339867 -0.00122252 -0.02431163 -0.05699017 -0.02632318 -0.03631433
 -0.03208023 -0.0474344  -0.02202571 -0.00951526]
Fitness de la mejor posición para 2 WeightedSphere: 0.07441556804904184
Mejor posición encontrada para 5 Schwefel1_2: [ 0.15354002 -0.44303122 -0.06294563  0.50304862 -0.42929777  0.36420201
 -0.01480569 -0.25899725  0.39016057  0.01493612]
Fitness de la mejor posición para 5 Schwefel1_2: 0.46746556319151866
Mejor posición encontrada para 12 salomon: [-0.05936881 -2.10066681 -0.64131853  0.01215724 -1.24896483 -1.82157143
 -0.57368512 -0.23411318 -2.11820422 -1.18364956]
Fitness de la mejor posición para 12 salomon: 0.3998748454994555
Mejor posición encontrada para 8 easom: [3.14165079 3.14163208 3.14160519 3.14162674 3.1416416  3.14161013
 3.14158973 3.14167021 3.14161121 3.14167619]
Fitness de la mejor posición para 8 easom: -0.9999999554001777
